## Autoencoder的代码实现

### 定义NN类和Autoencoder类

In [ ]:
import numpy as np
#import pandas as pd

#创建神经网络类
# nodes为1*n矩阵，表示每一层有多少个节点
class nn():
    def __init__(self,nodes):
        self.layers = len(nodes)
        self.nodes = nodes;
        # 学习率
        self.u = 1.0;
        # 权值
        self.W = list();
        # 偏差值
        self.B = list()
        # 层值
        self.values = list();
        # 误差
        self.error = 0;
        # 损失
        self.loss = 0;

        for i in range(self.layers-1):
            # 权值初始化，权重范围-0.5~0.5
            self.W.append(np.random.random((self.nodes[i],self.nodes[i+1])) - 0.5)   
            # B值初始化
            self.B.append(0)

        
        for j in range(self.layers):
            # values值初始化
            self.values.append(0)
            
            
#创建autoencoder类，可以看成是多个神经网络简单的堆叠而来
class autoencoder():
    def __init__(self):
        self.encoders = list()
    def add_one(self,nn):
        self.encoders.append(nn)

### 构建Autoencoder框架

In [ ]:
import numpy as np
from base import nn,autoencoder

# 激活函数
def sigmod(x):
    return 1.0 / (1.0 + np.exp(-x))

#前馈函数
def nnff(nn,x,y):
    layers = nn.layers
    numbers = x.shape[0]
    # 赋予初值
    nn.values[0] = x
    for i in range(1,layers):
        nn.values[i] = sigmod(np.dot(nn.values[i-1],nn.W[i-1])+nn.B[i-1])
    # 最后一层与实际的误差
    nn.error = y - nn.values[layers-1]
    nn.loss = 1.0/2.0*(nn.error**2).sum()/numbers
    return nn 

#BP函数
def nnbp(nn):
    layers = nn.layers;
    #初始化delta
    
    deltas = list();
    for i in range(layers):
        deltas.append(0)
    
    #最后一层的delta为
    deltas[layers-1] = -nn.error*nn.values[layers-1]*(1-nn.values[layers-1])
    #其他层的delta为
    for j in range(1,layers-1)[::-1]:#倒过来
        deltas[j] = np.dot(deltas[j+1],nn.W[j].T)*nn.values[j]*(1-nn.values[j])
    #更新W值
    for k in range(layers-1):
        nn.W[k] -= nn.u*np.dot(nn.values[k].T,deltas[k+1])/(deltas[k+1].shape[0])
        nn.B[k] -= nn.u*deltas[k+1]/(deltas[k+1].shape[0])
    return nn

#对神经网络进行训练
def nntrain(nn,x,y,iterations):
    for i in range(iterations):
        nnff(nn,x,y)
        nnbp(nn)
    return nn

#建立autoencoder框架
def aebuilder(nodes):
    layers = len(nodes)
    ae = autoencoder()
    for i in range(layers-1):
        ae.add_one(nn([nodes[i],nodes[i+1],nodes[i]]))    
    return ae
        
#训练autoencoder
def aetrain(ae,x,interations):
    elayers = len(ae.encoders)
    for i in range(elayers):
        #单层训练
        ae.encoders[i] = nntrain(ae.encoders[i],x,x,interations)
        #单层训练后，获取该层中间层的值，作为下一层的训练
        nntemp = nnff(ae.encoders[i],x,x)
        x = nntemp.values[1]   
    return ae  

### 验证结果

In [ ]:
import autoencoder
from base import autoencoder,nn
import numpy as np

x = np.array([[0,0,1,0,0],
            [0,1,1,0,1],
            [1,0,0,0,1],
            [1,1,1,0,0],
            [0,1,0,1,0],
            [0,1,1,1,1],
            [0,1,0,0,1],
            [0,1,1,0,1],
            [1,1,1,1,0],
            [0,0,0,1,0]])
y = np.array([[0],
            [1],
            [0],
            [1],
            [0],
            [1],
            [0],
            [1],
            [1],
            [0]])
#################################
# step1 建立autoencoder
#弄两层autoencoder
nodes=[5,3,2]
#建立auto框架
ae = autoencoder.aebuilder(nodes)
#设置部分参数
#训练
ae = autoencoder.aetrain(ae, x, 6000)
##############################
# step2 微调
#建立完全体的autoencoder
nodescomplete = np.array([5,3,2,1])
aecomplete = nn(nodescomplete)
for i in range(len(nodescomplete)-2):
    aecomplete.W[i] = ae.encoders[i].W[0]
    
aecomplete = autoencoder.nntrain(aecomplete, x, y, 6000)
print aecomplete.values[3]